In [1]:
# import dill
import scanpy as sc
import os
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np

# import pyranges
# Set stderr to null to avoid strange messages from ray
import sys
_stderr = sys.stderr
null = open(os.devnull,'wb')
from scipy.sparse import issparse


import itertools
import anndata
import itertools
import anndata
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import warnings
warnings.simplefilter(action='ignore')
import numpy as np
from sklearn.preprocessing import normalize

from itertools import product
import umap
from collections import Counter
from sklearn.decomposition import PCA
from gridot.utils import *
from gridot.train import *

In [2]:
rna_anndata = sc.read_h5ad('./data/adata.h5ad')

# Check if raw data exists
if rna_anndata.raw is not None:
    # Access the raw data
    raw_data = rna_anndata.raw.X
    raw_obs = rna_anndata.obs.copy()  # Cell metadata (unchanged)
    raw_var = rna_anndata.raw.var.copy()  # Gene metadata (original raw genes)

    # Create a new AnnData object using the raw data
    rna_anndata = sc.AnnData(X=raw_data, obs=raw_obs, var=raw_var)

    # Print the new raw AnnData object to verify
    print(rna_anndata)
else:
    print("The AnnData object does not contain raw data.")

AnnData object with n_obs × n_vars = 2664 × 21255
    obs: 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'ingest_celltype_label', 'leiden_res_0.8', 'celltype'
    var: 'gene_ids', 'feature_types', 'genome', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'


In [3]:
import scanpy as sc
import pandas as pd
import numpy as np

# Assuming your AnnData object is named `rna_anndata` and has a 'cell_type' column in .obs
groupby_key = 'celltype'

# Step 1: Perform differential expression
sc.tl.rank_genes_groups(rna_anndata, groupby=groupby_key, method='t-test')  # You can use 'wilcoxon' too

# Step 2: Extract DE results
de_results = rna_anndata.uns['rank_genes_groups']
groups = de_results['names'].dtype.names  # list of cell type names

top_genes_dict = {}

# Step 3: For each group (cell type), get top 25% DE genes
for group in groups:
    gene_names = np.array(de_results['names'][group])  # ranked gene names
    n_top = int(len(gene_names) * 0.25)  # top 25%
    top_genes = gene_names[:n_top]
    top_genes_dict[group] = top_genes

# Optional: Convert to DataFrame for easy inspection
top_genes_df = pd.DataFrame(dict([(k, pd.Series(v)) for k, v in top_genes_dict.items()]))

# Display or save
print(top_genes_df)


         B_cells CD4_T_cells CD8_T_cells CD14+_Monocytes Dendritic_cells  \
0           AFF3      BCL11B        CCL5          PLXDC2            CD74   
1           CD74       CAMK4        NKG7            FCN1         HLA-DRA   
2        HLA-DRA        LEF1        GZMA            DPYD            CST3   
3          BANK1        IL7R        IL32          SLC8A1        HLA-DRB1   
4        RALGPS2        TCF7       PRKCH           LRMDA        HLA-DPA1   
...          ...         ...         ...             ...             ...   
5308  AC124312.5       SOCS5     SNRNP27        WWC2-AS2      AL357079.1   
5309  AL121890.4      SEC22A      FER1L6      AL121987.2         CCDC146   
5310       WDR64     CARHSP1     SPINDOC            ELDR          LRRC72   
5311      UBE2J2       ACKR3       ACOT2         SULT1A4       LINC02021   
5312       BAMBI       NDST4  AC013437.1           CNTN4          ALKBH4   

     FCGR3A+_Monocytes    NK_cells  
0                 PSAP        GNLY  
1            

In [7]:

putative_pairs = pd.read_csv(f"./testing_results_pseudo/TF_motif/putative_TF_gene.tsv", sep='\t', index_col=0)

# Dictionary to store filtered pairs per cell type
filtered_pairs_by_celltype = {}

for cell_type, de_genes in top_genes_dict.items():
    # Convert list to set for faster lookup
    de_genes_set = set(de_genes)

    # Filter putative_pairs where Target is in DE gene list for that cell type
    filtered_pairs = putative_pairs[putative_pairs['TF'].isin(de_genes_set)]

    # Add extra columns
    filtered_pairs['rss_ratio'] = 1
    filtered_pairs['ftest'] = 1

    # Save to dictionary AFTER adding new columns
    filtered_pairs_by_celltype[cell_type] = filtered_pairs

    # Save each to file
    filtered_pairs.to_csv(f"./testing_results_pseudo/filtered_pairs_{cell_type}.tsv", sep='\t', index=False)


In [5]:
putative_pairs

,TF,Gene
0,ZNF251,A1BG-AS1
1,FOXI1,A1BG-AS1
2,ATF1,A1BG-AS1
3,CREB3L4,A1BG-AS1
4,PAX3,A1BG-AS1
...,...,...
1880495,SALL2,CWC25
1880503,XBP1,CWC25
1880506,TFE3,CWC25
1880524,NR2E1,CWC25
